In [15]:
# !pip install -r ./requirements.txt

In [18]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install aif360

In [6]:
# !pip install BlackBoxAuditing==0.1.54

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [4]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import DiabetesDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_model_params_for_mult_repair_levels

from source.preprocessing import get_preprocessor_for_diabetes
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_diabetes'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.2, 0.4]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'diabetes_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
    # 'session_uuid': str(uuid.uuid4()),
    'session_uuid': '163da20d-0491-493d-b718-b33de7b259fd',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  163da20d-0491-493d-b718-b33de7b259fd


## Initialize custom objects

In [9]:
data_loader = DiabetesDataset(dataset_path='./data/diabetes-clean.csv')
data_loader.categorical_columns = [col for col in data_loader.categorical_columns if col not in ('diag_1', 'diag_2', 'diag_3')]
data_loader.X_data.head()

,number_diagnoses,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,race,gender,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,9,3,59,0,18,0,0,0,Caucasian,Female,...,No,No,Up,No,No,No,No,No,Ch,Yes
1,9,3,31,6,16,0,0,0,Caucasian,Male,...,No,No,Steady,No,No,No,No,No,No,Yes
2,8,5,73,0,12,0,0,0,Caucasian,Male,...,No,No,No,No,No,No,No,No,No,Yes
3,9,9,47,2,17,0,0,0,AfricanAmerican,Female,...,No,No,Steady,No,No,No,No,No,No,Yes
4,7,7,62,0,11,0,0,0,AfricanAmerican,Male,...,No,No,Steady,No,No,No,No,No,Ch,Yes


In [10]:
data_loader.y_data.value_counts()

0    34649
1    11066
Name: readmitted, dtype: int64

In [11]:
data_loader.X_data.shape

(45715, 44)

In [12]:
data_loader.X_data[['diag_1', 'diag_2', 'diag_3']].isna().sum()

diag_1    0
diag_2    0
diag_3    0
dtype: int64

In [13]:
data_loader.X_data['race'].unique()

array(['Caucasian', 'AfricanAmerican', 'Other', 'Hispanic', 'Asian'],
      dtype=object)

In [14]:
data_loader.X_data['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [15]:
data_loader.y_data.value_counts()

0    34649
1    11066
Name: readmitted, dtype: int64

In [20]:
from virny.preprocessing.basic_preprocessing import preprocess_dataset
from source.preprocessing import remove_disparate_impact

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_data_loader.categorical_columns = [col for col in exp_iter_data_loader.categorical_columns if col not in ('gender', 'race')]
exp_iter_data_loader.X_data['race_binary'] = exp_iter_data_loader.X_data['race'].apply(lambda x: 1 if x == 'Caucasian' else 0)
exp_iter_data_loader.full_df = exp_iter_data_loader.full_df.drop(['gender', 'race'], axis=1)
exp_iter_data_loader.X_data = exp_iter_data_loader.X_data.drop(['gender', 'race'], axis=1)

column_transformer = get_preprocessor_for_diabetes(exp_iter_data_loader)
init_base_flow_dataset = preprocess_dataset(data_loader, column_transformer, TEST_SET_FRACTION, dataset_split_seed=100)
init_base_flow_dataset.X_train_val['race_binary'] = exp_iter_data_loader.X_data.loc[init_base_flow_dataset.X_train_val.index, 'race_binary']
init_base_flow_dataset.X_test['race_binary'] = exp_iter_data_loader.X_data.loc[init_base_flow_dataset.X_test.index, 'race_binary']

In [21]:
base_flow_dataset = remove_disparate_impact(init_base_flow_dataset, alpha=0.6)

In [22]:
base_flow_dataset.X_test.head(5)

,cat__age_[0-10),cat__age_[10-20),cat__age_[20-30),cat__age_[30-40),cat__age_[40-50),cat__age_[50-60),cat__age_[60-70),cat__age_[70-80),cat__age_[80-90),cat__age_[90-100),...,enc__diag_2,enc__diag_3,num__number_diagnoses,num__time_in_hospital,num__num_lab_procedures,num__num_procedures,num__num_medications,num__number_outpatient,num__number_emergency,num__number_inpatient
21652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,336.0,401.0,0.165281,1.472831,1.745569,0.438222,0.062790,-0.323894,-0.247384,0.038980
40167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,110.0,240.0,0.165281,-0.192041,-0.099506,-0.762203,-1.099834,-0.323894,-0.247384,-0.603038
6145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,252.0,229.0,0.165281,1.472831,1.694317,-0.161990,0.708691,-0.323894,-0.247384,-0.603038
29567,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,574.0,18.0,-0.994954,-1.190965,-0.765783,-0.762203,-0.970654,-0.323894,-0.247384,-0.603038
6210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,539.0,333.0,0.165281,3.137704,1.745569,2.839073,2.904758,-0.323894,-0.247384,-0.603038


In [23]:
init_base_flow_dataset.X_test.head(5)

,cat__age_[0-10),cat__age_[10-20),cat__age_[20-30),cat__age_[30-40),cat__age_[40-50),cat__age_[50-60),cat__age_[60-70),cat__age_[70-80),cat__age_[80-90),cat__age_[90-100),...,enc__diag_3,num__number_diagnoses,num__time_in_hospital,num__num_lab_procedures,num__num_procedures,num__num_medications,num__number_outpatient,num__number_emergency,num__number_inpatient,race_binary
21652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,418,0.745398,1.472831,2.001830,0.438222,0.450331,0.340975,-0.247384,0.038980,1
40167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,244,0.745398,-0.192041,0.054250,-0.762203,-0.970654,-0.323894,-0.247384,-0.603038,1
6145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,231,0.745398,1.472831,1.950578,-0.161990,1.225413,-0.323894,-0.247384,-0.603038,1
29567,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,28,-0.994954,-1.190965,-0.765783,-0.762203,-0.970654,-0.323894,-0.247384,-0.603038,0
6210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,346,0.745398,3.137704,2.001830,2.839073,3.809020,-0.323894,-0.247384,-0.603038,1


In [68]:
data_loader.X_data.loc[init_base_flow_dataset.X_test.index, :].head()

,number_diagnoses,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,race,gender,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
21652,9,9,83,2,20,1,0,1,Caucasian,Female,...,No,No,No,No,No,No,No,No,No,Yes
40167,9,4,45,0,9,0,0,0,Caucasian,Female,...,No,No,Steady,No,No,No,No,No,No,Yes
6145,9,9,82,1,26,0,0,0,Caucasian,Female,...,No,No,Down,No,No,No,No,No,Ch,Yes
29567,6,1,29,0,9,0,0,0,Hispanic,Female,...,No,No,Steady,No,No,No,No,No,No,Yes
6210,9,14,83,6,46,0,0,0,Caucasian,Male,...,No,No,Steady,No,No,No,No,No,No,Yes


## Run experiment iterations

### Experiment iteration 1

In [31]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [113]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [114]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-08-04 01:29:54 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0, 0.2, 0.4]',
 'intervention_param': 0.4,
 'model_init_seed': 100,
 'session_uuid': '163da20d-0491-493d-b718-b33de7b259fd'}




2023-08-04 01:29:54 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([28666, 33065, 17959, 24861,    34, 17810, 34743,  4256, 16528,
            34310, 17161, 15937, 10740, 19844,  5115, 14586, 11307,  8230,
            33832, 19081],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([28666, 33065, 17959, 24861,    34, 17810, 34743,  4256, 16528,
            34310, 17161, 15937, 10740, 19844,  5115, 14586, 11307,  8230,
            33832, 19081],
           dtype='int64')


Multiple alphas:   0%|          | 0/3 [00:00<?, ?it/s]

...

Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/20 [00:00<?, ?it/s]

intervention_param:  0.2


KeyboardInterrupt: 

### Experiment iteration 2

In [44]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [45]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-07-31 14:43:04 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4]',
 'model_init_seed': 200,
 'session_uuid': '116104d6-7d63-4564-ae57-82b19b260fa5'}




2023-07-31 14:43:04 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')


Multiple alphas:   0%|          | 0/5 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.4
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_pubcov_CA/tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [46]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)